In [34]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer


In [35]:
# llama
model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
local_path = '/nfs/public/hf/models/'
full_model_path = local_path + model

In [36]:
# model = 'microsoft/Phi-3.5-mini-instruct'
model = 'google/gemma-2-9b-it'
full_model_path = model

In [37]:
system_content = [
    'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.',
    'Now perform the steps in the plan you created. Use the most precise, accurate and up-to-date information available. To save space, be very concise when describing the actions. Conclude by stating the answer that you reached by following the steps you outlined.',
]
temperature = 0.2
dataset = pd.read_json('../resources/data/Franklin/full_study.jsonl', lines=True)
df = dataset[['internal_id', 'question']].sample(1)
# dataset = pd.read_json('../resources/data/StrategyQA/dev.jsonl', lines=True)
# df = dataset[['qid', 'question']].sample(1, random_state=43)

In [38]:
tokenizer = AutoTokenizer.from_pretrained(
    full_model_path,
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    full_model_path,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

# model.config.use_flash_attention = True

device = model.device
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.02s/it]


In [39]:
def create_conversation(question):
    """Create a conversation with the system and user."""
    return [
        {'role': 'system', 'content': system_content[0]},
        {'role': 'user', 'content': question},
    ]


df['conversations'] = df.apply(lambda x: create_conversation(x['question']), axis=1)

In [40]:
responses = []
conversations = df['conversations'].tolist()
# each conversation looks like:
# [
#     {'role': 'system', 'content': config.system_content[0]},
#     {'role': 'user', 'content': i},
# ]

# we want them to look like
# [
#     {'role': 'system', 'content': config.system_content[0]},
#     {'role': 'user', 'content': i},
#     {'role': 'assistant', 'content': '...'},
#     {'role': 'user', 'content': 'Now perform the steps in the plan you created.'},
#     {'role': 'assistant', 'content': '...'},
# ]

for conversation in tqdm(conversations, desc='Generating responses'):
    history = conversation
    # inputs = tokenizer.apply_chat_template(
    #     history,
    #     padding=True,
    #     return_tensors='pt',
    #     add_generation_prompt=True,
    # ).to(device)

    inputs = tokenizer(
        ', '.join(str(h) for h in history),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt',
    ).to(device)['input_ids']

    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=temperature,
    )

    history.append({'role': 'assistant', 'content': tokenizer.decode(outputs[0], skip_special_tokens=True)})

    # add a user message to prompt the assistant to perform the steps in the plan
    history.append({'role': 'user', 'content': system_content[1]})

    # generate the response
    inputs = tokenizer(
        ', '.join(str(h) for h in history),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt',
    ).to(device)['input_ids']

    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=temperature,
    )

    history.append({'role': 'assistant', 'content': tokenizer.decode(outputs[0], skip_special_tokens=True)})
    responses.append(history)

df['response'] = responses
df['history'] = [history]

Generating responses: 100%|██████████| 1/1 [00:21<00:00, 21.23s/it]


In [41]:
print(df['response'][df.index[0]])

[{'role': 'system', 'content': 'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.'}, {'role': 'user', 'content': 'For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?'}, {'role': 'assistant', 'content': "{'role': 'system', 'content': 'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.'}, {'role': 'user', 'content': 'For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?'}\n\nHere is a step-by-step plan to find the answer:\n\n1. **Identify the Eastern European country with the lowest population in 2010.**\n    * Consult a re

In [47]:
meta_response = df['response'][df.index[0]][2]['content']
object_response = df['response'][df.index[0]][4]['content']

prefix = f'{system_content[0]} {df["question"][df.index[0]]}'
meta_response = meta_response[len(prefix) :].strip()

# prefix += f' {system_content[0]} {df["question"][df.index[0]]} {system_content[1]} {meta_response}'


df['history'][df.index[0]]

[{'role': 'system',
  'content': 'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.'},
 {'role': 'user',
  'content': 'For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?'},
 {'role': 'assistant',
  'content': "{'role': 'system', 'content': 'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.'}, {'role': 'user', 'content': 'For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?'}\n\nHere is a step-by-step plan to find the answer:\n\n1. **Identify the Eastern European country with the lowest population in 2010.**\n    * Cons

In [ ]:
object_response = object_response[len(prefix) :].strip()

In [43]:
print(df['question'][df.index[0]])

For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?


In [44]:
print(meta_response)

population in 2010, what will be its rural population in 2030?'}

Here is a step-by-step plan to find the answer:

1. **Identify the Eastern European country with the lowest population in 2010.**
    * Consult a reliable source of demographic data for 2010, such as the World Bank or the United Nations.
    * Filter the data to focus on Eastern European countries.
    * Determine the country with the smallest population.
2. **Find population projections for that country.**
    * Search for population forecasts or projections for the identified country, specifically for the year 2030.
    * Look for projections that break down the population into urban and rural categories.
3. **Extract the projected rural population for 2030.**
    * Locate the projected rural population figure for 2030 within the obtained data.


In [45]:
print(object_response)

population in 2010, what will be its rural population in 2030?'}, {'role': 'assistant', 'content': "{'role': 'system', 'content': 'Create a step-by-step plan for finding the answer to the following problem. Do not answer the question. Do not perform the actions in the plan. Your only task is to outline the steps involved in a concise and clear manner.'}, {'role': 'user', 'content': 'For the country in Eastern Europe that had the lowest population in 2010, what will be its rural population in 2030?'}\n\nHere is a step-by-step plan to find the answer:\n\n1. **Identify the Eastern European country with the lowest population in 2010.**\n    * Consult a reliable source of demographic data for 2010, such as the World Bank or the United Nations.\n    * Filter the data to focus on Eastern European countries.\n    * Determine the country with the smallest population.\n2. **Find population projections for that country.**\n    * Search for population forecasts or projections for the identified co